In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy as sc

## Importing all the data

In [2]:
dire0="C:/Users/HP/Desktop/EPT/3e/PFE/Work/Renzo_material/Wind_data/Table_Complete.csv"

In [3]:
os.chdir("C:\\Users\\HP\\Desktop\\EPT\\3e\\PFE\\Work\\Wind_project\\Renzo_material\\Wind_data")

In [4]:
all_data=pd.read_csv("Table_Complete.csv",encoding='utf-8')

In [5]:
def data(beg,end):
    l=[col[0]]
    l.extend(col[beg:end])
    return(all_data[l])

In [6]:
col=all_data.columns.to_list()

In [7]:
time=data(1,146)

In [8]:
forecast=data(146,291)

In [9]:
adme=data(580,725)

## Forecast P epsilon

In [10]:
eps=0.018

In [11]:
def pe(eps,forecast):
    col_for=forecast.columns.to_list()
    forecast_eps=forecast.copy()
    for i in col_for[1:]:
        forecast_eps.loc[forecast_eps[i]<eps,i]=eps
        forecast_eps.loc[forecast_eps[i]>=(1-eps),i]=1-eps
    return(forecast_eps)

In [12]:
forecast_eps=pe(eps,forecast)

In [13]:
def pe_dot(forecast,dt):
    col=forecast.columns.to_list()
    forecast_dot=pd.DataFrame(columns=col)
    forecast_dot['Date']=forecast['Date']
    del col[0]
    forecast_dot[col]=(forecast[col].shift(-1)-forecast[col])/dt
    forecast_dot=forecast_dot.drop(forecast_dot.index[len(forecast_dot)-1])
    return(forecast_dot)

In [14]:
dt=0.05

In [15]:
forecast_dot=pe_dot(forecast_eps,dt)

## Error

In [16]:
col_err=['Date']
l=['error_'+str(i) for i in range(1,146)]
col_err.extend(l)
error=pd.DataFrame(columns=col_err)
col_adme=adme.columns.to_list()
col_for=forecast.columns.to_list()
del col_adme[0]
del col_for[0]
error['Date']=adme['Date']
for i in range(len(l)):
    error[l[i]]=adme[col_adme[i]]-forecast_eps[col_for[i]]

# Moment Matching

## First moment

In [18]:
def first_moment(v,theta1,theta2,dt,N):
    if theta1==theta2:
        return(v*np.exp(-theta1*dt))
    else:
        m=np.zeros(N)
        m[0]=v
        ds=dt/N
        for i in range(1,N):
            m[i]=m[i-1]*(1-theta1+(theta2-theta1)*(i/N)*ds)
    return(m)

## Second moment

In [22]:
def second_moment(v,theta1,theta2,p1,p2,alpha,dt,N):
    m1=first_moment(v,theta1,theta2,dt,N)
    m2=np.zeros(N)
    m2[0]=v**2
    ds=dt/N
    if theta1!=theta2:
        for i in range(1,N):
            m1=m1*np.exp((i-1)/N)
            p=p1+(p2-p1)*((i-1)/N)
            m2[i]=m2[i-1]+2*theta1*ds*(-(1+alpha)*m2[i-1])+alpha*m1*(1-2*p)+alpha*p*(2-p)
    else:
        for i in range(1,N):
            theta=theta1+(theta2-theta1)*((i-1)/N)
            m1=m1*np.exp((i-1)/N)
            p=p1+(p2-p1)*((i-1)/N)
            m2[i]=m2[i-1]+2*theta1*ds*(-(1+alpha)*m2[i-1])+alpha*m1*(1-2*p)+alpha*p*(2-p)

## Density approximation parameters

In [23]:
def moment_matching(m1,m2):
    mu=m1
    sig2=m2-m1**2
    xi1=-((mu+1)*(mu**2+sig2-1))/(2*sig2)
    xi2=((mu-1)*(mu**2+sig2-1))/(2*sig2)
    return(xi1,xi2)

# Initial guess

In [20]:
M,N=forecast.shape
N-=1
theta1=0.1
theta2=0.2

In [21]:
def initial_guess(adme,M,N,dt):
    estimator=0
    

In [ ]:
def initial_theta():
    

In [ ]:
def initial_theta0():
    